In [ ]:
# https://www.kaggle.com/code/hassanamin/pyspark-customer-churn-prediction/notebook
# https://medium.com/swlh/churn-prediction-using-pyspark-a1f4ef0439b3
#https://www.kaggle.com/code/tylerx/machine-learning-with-spark/notebook
#https://www.kaggle.com/code/fatmakursun/pyspark-ml-tutorial-for-beginners/notebook    

In [ ]:
a) Exploratory Data Analysis
In this section, perform both a visual and statistical exploratory analysis to gain
insights about the dataset.
c) Data Cleaning and Feature Selection
In this section, perform data pre-processing and feature selection for the model,
which you will build in the next section.
d) Model Building
In this section, use the pre-processed data and the selected features to build a
‘decision-tree’ model to predict customer churn.

In [69]:
from pyspark.sql import SparkSession

In [70]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [71]:
data = spark.read.csv('churn (1).csv',inferSchema=True, header=True)

In [72]:
data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [73]:
data.describe().show()

+-------+----------+------+------------------+----------+------------------+------------+-------------+------------+----------------+-----------+---------------+--------------+----------------+--------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Dependents|            tenure|PhoneService|MultipleLines|OnlineBackup|DeviceProtection|TechSupport|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|      TotalCharges|Churn|
+-------+----------+------+------------------+----------+------------------+------------+-------------+------------+----------------+-----------+---------------+--------------+----------------+--------------------+------------------+-----+
|  count|      7043|  7043|              7043|      7043|              7043|        7043|         7043|        7043|            7043|       7043|           7043|          7043|            7043|                7043|              7043| 7043|
|   mean|      null|  null|0.16214681243

In [74]:
data.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'TotalCharges',
 'Churn']

In [ ]:
customerID,MonthlyCharges, OnlineSecurity, StreamingTV, InternetService and Partner

In [75]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,MinMaxScaler,StringIndexer

In [76]:
import pandas as pd

In [77]:
d = pd.read_csv('churn (1).csv')

In [78]:
d.head()

,customerID,gender,SeniorCitizen,Dependents,tenure,PhoneService,MultipleLines,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
0,7590-VHVEG,Female,0,No,1,No,No phone service,Yes,No,No,No,Month-to-month,Yes,Electronic check,29.85,No
1,5575-GNVDE,Male,0,No,34,Yes,No,No,Yes,No,No,One year,No,Mailed check,1889.5,No
2,3668-QPYBK,Male,0,No,2,Yes,No,Yes,No,No,No,Month-to-month,Yes,Mailed check,108.15,Yes
3,7795-CFOCW,Male,0,No,45,No,No phone service,No,Yes,Yes,No,One year,No,Bank transfer (automatic),1840.75,No
4,9237-HQITU,Female,0,No,2,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,151.65,Yes


In [79]:
cat_col = [i[0] for i in data.dtypes if i[1]=='string'][1:]

In [80]:
cat_col

['gender',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'TotalCharges',
 'Churn']

In [88]:
strIdx = StringIndexer(inputCol = 'gender', outputCol = "genderIdx")

In [98]:
# strIdx = []
# for col in cat_col:
strIdx = StringIndexer(inputCol = 'gender', outputCol = "genderIdx")

In [99]:
catVect  = VectorAssembler(inputCols=['genderIdx'], outputCol='features')

In [100]:
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")

In [101]:
num_col = [i[0] for i in data.dtypes if i[1]!='string']

In [102]:
num_col

['SeniorCitizen', 'tenure']

In [103]:
numVect = VectorAssembler(inputCols = num_col, outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

In [104]:
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")

In [105]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [106]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)

In [107]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("Churn", "trueLabel")

In [110]:
data

DataFrame[customerID: string, gender: string, SeniorCitizen: int, Dependents: string, tenure: int, PhoneService: string, MultipleLines: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, TotalCharges: string, Churn: string]

In [115]:
data = data.withColumn("TotalCharges",data.TotalCharges.cast('float'))

In [116]:
data.dtypes

[('customerID', 'string'),
 ('gender', 'string'),
 ('SeniorCitizen', 'int'),
 ('Dependents', 'string'),
 ('tenure', 'int'),
 ('PhoneService', 'string'),
 ('MultipleLines', 'string'),
 ('OnlineBackup', 'string'),
 ('DeviceProtection', 'string'),
 ('TechSupport', 'string'),
 ('StreamingMovies', 'string'),
 ('Contract', 'string'),
 ('PaperlessBilling', 'string'),
 ('PaymentMethod', 'string'),
 ('TotalCharges', 'float'),
 ('Churn', 'string')]

In [108]:
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr])

In [109]:
piplineModel = pipeline.fit(train)

IllegalArgumentException: Output column features already exists.